In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# print('Libraries imported.')

In [2]:
CLIENT_ID = 'V5WOIXUCWLPIGI1SNR3UY0WL2Y4JLPTNXOTL4X1UN2DWG3WI' # your Foursquare ID
CLIENT_SECRET = 'AUA4DRJPYKXXG3JWFUZRI1JJ5UCEGBEUJZQXHUT1AQYMZLPG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [3]:
cities = {"Boston, MA", "New York, NY", "Chicago, IL", "San Francisco, CA", "Dallas, TX"}
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d142941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 50")

Total number of pizza places in Boston, MA =  282
Showing Top 50
Total number of pizza places in New York, NY =  317
Showing Top 50
Total number of pizza places in Dallas, TX =  264
Showing Top 50
Total number of pizza places in San Francisco, CA =  292
Showing Top 50
Total number of pizza places in Chicago, IL =  295
Showing Top 50


In [6]:
maps["Boston, MA"]

In [7]:
maps["New York, NY"]

In [8]:
maps["Chicago, IL"]

In [9]:
maps["San Francisco, CA"]

In [10]:
maps["Dallas, TX"]

In [11]:
maps = {}
for x in cities:
    city_latitude = np.mean([results[x]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[x]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_longitude = np.mean([results[x]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[x]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[x] = folium.Map(location=[city_latitude, city_longitude], zoom_start=8)
    mean_venue = [df_venues[x]['Lat'].mean(), df_venues[x]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[x]['Lat'], df_venues[x]['Lng'], df_venues[x]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[x])
        folium.PolyLine([mean_venue, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[x])
    
    label = folium.Popup("Mean Location", parse_html=True)
    folium.CircleMarker(
        mean_venue,
        radius=9,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186bb',
        fill_opacity=0.9,
        parse_html=False).add_to(maps[x])

    print(x)
    print("Average Distance from Mean Venus coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - mean_venue),1,df_venues[x][['Lat','Lng']].values)))

Boston, MA
Average Distance from Mean Venus coordinates
0.02125993287090133
New York, NY
Average Distance from Mean Venus coordinates
0.02062215773496001
Dallas, TX
Average Distance from Mean Venus coordinates
0.1045204947760041
San Francisco, CA
Average Distance from Mean Venus coordinates
0.0320821055785922
Chicago, IL
Average Distance from Mean Venus coordinates
0.046745532353904454


In [12]:
maps["Boston, MA"]

In [13]:
maps["New York, NY"]

In [14]:
maps["Chicago, IL"]

In [15]:
maps["San Francisco, CA"]

In [16]:
maps["Dallas, TX"]